In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("tickets.csv")
df.head(2)

,ticket_id,client,service,logged_date,status,time_spent,assignee,logger
0,1000,B,F1,2024-03-31 15:54:00 UTC,awaiting_response,8,member4,user2
1,1001,A,C1,2024-10-23 13:53:00 UTC,assigned,8,member4,user3


In [3]:
df.dtypes

ticket_id       int64
client         object
service        object
logged_date    object
status         object
time_spent      int64
assignee       object
logger         object
dtype: object

In [4]:
df['logged_date'] = pd.to_datetime(df['logged_date'], format="%Y-%m-%d %H:%M:%S UTC")
df.dtypes

ticket_id               int64
client                 object
service                object
logged_date    datetime64[ns]
status                 object
time_spent              int64
assignee               object
logger                 object
dtype: object

In [5]:
df = df.sort_values(by='logged_date').reset_index(drop=True)
df.head()

,ticket_id,client,service,logged_date,status,time_spent,assignee,logger
0,1588,E,E1,2024-01-01 03:35:00,awaiting_response,5,member2,user1
1,1724,E,A1,2024-01-01 10:29:00,closed,1,member2,user1
2,1499,D,B1,2024-01-01 10:42:00,closed,7,member3,user4
3,1256,D,B1,2024-01-01 21:54:00,assigned,2,member3,user3
4,1254,D,F1,2024-01-02 01:04:00,awaiting_response,6,member4,user1
